## Was ist RAG?

##### 1. **Retrieve**: Finde die relevantesten Dokumente zur Beantwortung der Frage.
##### 2. **Augment**: Gib die relevanten Dokumente zusammen mit der Frage an das LLM weiter.
##### 3. **Generate**: Das LLM generiert eine Antwort *basierend* auf den retrievted Dokumenten.


In [2]:
# Imports & keys 

import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

os.chdir("/workspace")

# initialize API keys
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"API Keys retrieved successfully")
else:
    print("API Keys not found")

API Keys retrieved successfully


## Was ist ein *embedding*?

### Worte werden in Zahlen umgewandelt

- Das **Embedding Modell** konvertiert einen Begriff oder einen Satz in eine Liste von Zahlen (-> VeKtor).  
- Vektoren (von Begriffen oder Sätzen) die in eine *ähnliche* Richtung zeigen, repräsentieren Begriffe mit einer *ähnlichen* semantischen Bedeutung.

In [91]:
# define a small embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# examples of sentences with different semantic meanings
# sentences = [
#     "Es regnet draußen",                    
#     "Ich brauche einen Regenschirm",        
#     "Die Hauptstadt von Frankreich ist Paris",
# ]
sentences = [
    "Hund",                    
    "Wolf",        
    "Regenschirm",
]

# embed the sentences using the same embedding model
embeddings = [embedding_model.embed_query(s) for s in sentences]

In [92]:
# this is how an embedding looks like
print(f"This is the sentence:", sentences[0])
print("These are the first 5 numbers of the vector:", embeddings[0][:5], "...")
print(f"The corresponding vector has a length of: {len(embeddings[0])} dimensions")


This is the sentence: Hund
These are the first 5 numbers of the vector: [-0.040633514523506165, -0.04273275285959244, 0.002069076057523489, 0.015406472608447075, 0.01820545643568039] ...
The corresponding vector has a length of: 1536 dimensions


In [93]:
# calculate semantic proximity using cosine_similarity
sim_matrix = cosine_similarity(np.array(embeddings))

# print relationshop between the sentences
print("Semantische Ähnlichkeit zwischen folgenden Sätzen:")
# labels = [
#     '"Es regnet draußen" vs. "Ich brauche einen Regenschirm"', 
#     '"Es regnet draußen" vs. "Die Hauptstadt von Frankreich ist Paris"',
#     '"Ich brauche einen Regenschirm" vs. "Die Hauptstadt von Frankreich ist Paris"'
# ]
labels = [
    '"Hund" vs. "Wolf', 
    '"Hund" vs. "Regenschirm"',
    '"Wolf" vs. "Regenschirm"'
]
    
scores = [
    sim_matrix[0, 1],
    sim_matrix[0, 2],
    sim_matrix[1, 2],
]
for lbl, sc in zip(labels, scores):
    print(f"{lbl:<6}: {sc: .3f}")

Semantische Ähnlichkeit zwischen folgenden Sätzen:
"Hund" vs. "Wolf:  0.385
"Hund" vs. "Regenschirm":  0.289
"Wolf" vs. "Regenschirm":  0.154


## Was ist ein *Vector Store*?

- Ein einzelnes Embedding ist nur eine Zahlentabelle. Erst wenn wir **tausende** davon nebeneinander speichern, können wir nach “welche klingen am ähnlichsten?” fragen.  
- Der Vektorstore speichert jeden Vektor samt Metadaten (Titel, URL, Abschnitt) **und** baut einen Index für schneller Ähnlichkeitssuche (≈ Millisekunden).  
- Ohne diesen Index müssten wir bei jeder Frage alle Vektoren einzeln vergleichen – bei > 5000 Artikeln wäre das Minuten statt Millisekunden.  
- Der Vector Store ist die **semantische Stichwortkartei** – er liefert die x passendsten Text‑Schnipsel, die wir anschließend im 1. Schritt RAG‑Prozesses **retrieven**.


In [6]:
# show the content of the data path
data_path="data/example_data"

for datei in os.listdir(data_path):
    if datei.endswith(".txt"):
        print(repr(datei))

'10_Copernicus-Report_ 2024 war 1_6 Grad wärmer als vo.txt'
'07_Dunkelflauten_ Wie hohe Strompreise gesenkt werden.txt'
'40_FDA lässt neuartiges Medikament zur Behandlung von.txt'
'33_KI-Modell analysiert und generiert DNA-Strukturen.txt'
'16_KI senkt Zustimmung zu Verschwörungstheorien.txt'


In [8]:
# build the vectorstore 

# --- set parameters ---
data_path = "data/example_data"  # data input folder
text_chunk_size = 512  # chunk size
text_chunk_overlap = 20  # chunk overlap
embedding_model = "text-embedding-3-small"  # embedding model
save_path = "data/vectorstores"  # directory to save the vector store

"""
Creates a vectorstore for both tables and text documents.

Parameters:
    data_table(str): path to folder containing txt files
    text_chunk_size (int): Chunk size for text documents. Default is 512.
    text_chunk_overlap (int): Overlap for text documents. Default is 20.
    embedding_model (str): Embedding model to use. Default is "text-embedding-3-small".
    db_backend (str): Vector store backend. Default is "faiss".
    save_path (str): Directory to save the vector store. Defaults to "vectorstores".

Returns:
    vectorstore: The created vector store.
"""

# ensure save directory exists
os.makedirs(save_path, exist_ok=True)

# Read all txt files and create documents
documents = []
for filename in os.listdir(data_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(data_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Split and get title
        lines = content.split('\n')
        title = lines[0].replace('TITLE: ', '').strip()

        # Get text content excluding title and empty line
        text_content = '\n'.join(lines[2:])

        # Create document with metadata
        doc = Document(
            page_content=text_content,
            metadata={'title': title}
        )
        documents.append(doc)

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=text_chunk_size,
    chunk_overlap=text_chunk_overlap
)

# Split text documents into overlapping chunks
split_documents = text_splitter.split_documents(documents)

# Create embeddings
embeddings = OpenAIEmbeddings(model=embedding_model)

# Build FAISS vectorstore and save to disk
vectorstore = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings
)
#vectorstore.save_local(save_path)

print(f"Vectorstore successfully created and saved to {save_path}")

Vectorstore successfully created and saved to data/vectorstores


In [12]:
# quick sanity check (retrieve relevant documents for a search query)
search_query = "Welche Medikamente werden gegen Schizophrenie eingesetzt?"
results = vectorstore.similarity_search(query=search_query, k=3)

# pretty print
for i, doc in enumerate(results, start=1):
    print(f"\n--- Result {i} ---")
    print(f"Title: {doc.metadata.get('title')}")
    print(f"Content:\n{doc.page_content[:400]}...")  


--- Result 1 ---
Title: FDA lässt neuartiges Medikament zur Behandlung von Schizophrenie zu
Content:
„Obwohl die Schizophrenie eine der sozioökonomisch teuersten Erkrankung mit einer Lebenszeitreduktion von mehr als 15 Jahren und einer massiven Einschränkung der Lebensqualität ist, findet leider nur wenig innovative Arzneiforschung statt. Erfreulicherweise ändert sich das im Moment. Andere vielversprechende Medikamente sind auf jeden Fall Emraclidin, welches als positiv allosterischer Modulator a...

--- Result 2 ---
Title: FDA lässt neuartiges Medikament zur Behandlung von Schizophrenie zu
Content:
„Zudem ist es prinzipiell gut, wenn Behandelnde aus einem breiten Spektrum an guten Medikamenten zur Behandlung der Patienten wählen können. Je nach Person und Krankheitsbild können unterschiedliche Medikamente besser passen. Dies ist seit einigen Jahrzehnten der erste neue Ansatz für ein Schizophrenie-Medikament mit einer offenbar guten Wirksamkeit und anderen Nebenwirkungen. Ob KarXT die